## Naive Bayes from Scratch Toy Example
<p>This is a toy example using Naive Bayes. The program below is not novel, but this is a great way to learn how to do machine learning without additional modules. I mean, we do want to learn how this thing works, right?</p>
<p>Let's play!</p>
<p>Let's say we have four people who reviewed movies, the liked and disliked marked by 0 for dislike and 1 for like as well as Y for like and N for dislike of a target movie. Then we get a fifth person who has given likes and dislikes for each of the three movies, but we don't know what that person's review for a target movie is. We want to predict whether that fifth movie reviewer will give Y or N for a target movie; thus we use Naive Bayes from scratch--no modules such as Sklearn and such--because we want to learn how Naive Bayes ticks.</p>
<p>I try to explain in plain English how this works from scratch as a layman, so, hopefully, others might enjoy grasping how Naive Bayes actually works like me.</p>

In [1]:
import numpy as np

"""
In the array for X_train, each list contains a 1 for like and 0 for dislike for some movies, which are features to predict whether that person likes the target movie indicated with Y or N in the list Y_train.
We are given a fifth person's likes and dislikes for some movies, and we will use those features to predict whether that person will like the target movie.
We already have our toy data divided up into training and testing sets, so let's go.
"""

X_train = np.array([
    [0, 1, 1],
    [0, 0, 1],
    [0, 0, 0],
    [1, 1, 0]])

Y_train = ['Y', 'N', 'Y', 'Y']

X_test = np.array([[1, 1, 0]])

In [2]:
def get_label_indices(labels):
    
    """
    We will use defaultdict to avoid getting key errors when different keys have the same values.
    We cannot avoid key errors using regular dictionaries in python, so we must use defaultdict instead.
    """
    
    from collections import defaultdict
    label_indices = defaultdict(list)
    for index, label in enumerate(labels):
        label_indices[label].append(index)
    return label_indices

In [3]:
label_indices = get_label_indices(Y_train)
print('label_indices:\n', label_indices)

label_indices:
 defaultdict(<class 'list'>, {'Y': [0, 2, 3], 'N': [1]})


In [4]:
def get_prior(label_indices):
    
    """
    Basically, this computes the prior by P(Y) and P(N).
    """
    
    prior = {label: len(indices) for label, indices in label_indices.items()}
    total_count = sum(prior.values())
    for label in prior:
        prior[label] /= total_count
    return prior

In [5]:
"""
We need to compute the prior assumption of Y and N.
P(Y) = 3/4 because there is Y, Y, Y, and N, making three Y's out of four targets.
P(N) = 1/4 because there is only one N out of four targets.
"""

prior = get_prior(label_indices)
print('Prior:', prior)

Prior: {'Y': 0.75, 'N': 0.25}


In [6]:
def get_likelihood(features, label_indices, smoothing=0):
    
    """
    The likelihood is a conditional probability in which P(feature|class).
    """
    
    likelihood = {}
    for label, indices in label_indices.items():
        likelihood[label] = features[indices, :].sum(axis=0) + smoothing
        total_count = len(indices)
        likelihood[label] = likelihood[label] / (total_count + 2 * smoothing)
    return likelihood

In [7]:
"""
          m1, m2, m3
person 1: [0, 1, 1] = Y
person 2: [0, 0, 1] = N
person 3: [0, 0, 0] = Y
person 4: [1, 1, 0] = Y

m1=f1,m2=f2,m3=f3

For f1, we have 1 like out of 0, 0, 1 for Y.
We have +1 for smoothing in the numerator and +2 in the denominator (i.e., two possible outcomes being Y and N).
Therefore P(f1/N) = 0 likes + 1 smoothing / 1 data point + 2 possibilities = 1/3 = 0.33333333333
P(f1/Y) = 1 like + 1 smoothing / 3 data points + 2 possibilities = 2/5 = 0.4
P(f2/N) = 0 likes + 1 smoothing / 1 data point + 2 possibilities = 1/3 = 0.33333333333
P(f2/Y) = 2 likes + 1 smoothing / 3 data points + 2 possibilities = 3/5 = 0.6
P(f3/N) = 1 like + 1 smoothing / 1 data point + 2 possibilities = 2/3 = 0.66666667
P(f3/Y) = 1 like + 1 smoothing / 3 data points + 2 possibilities = 2/5 = 0.4

          m1, m2, m3
person 5: [1, 1, 0] = ?
Now, will person 5 like or dislike a target movie, indicated with Y or N?
This is what we will use Naive Bayes for to answer this question.
"""

smoothing = 1
likelihood = get_likelihood(X_train, label_indices, smoothing)
print('Likelihood:\n', likelihood)

Likelihood:
 {'Y': array([0.4, 0.6, 0.4]), 'N': array([0.33333333, 0.33333333, 0.66666667])}


In [8]:
def get_posterior(X, prior, likelihood):
    """
    P(N)*P(f1=1|N)*P(f2=1|N)*P(f3=0|N) = P(N|x of X_train)
    P(Y)*P(f1=1|Y)*P(f2=1|Y)*P(f3=0|Y) = P(Y|x of X_train)
    
    P(Y|x of X_train) + P(N|x of X_train) = sum
    
    P(Y|x of X_train) / sum = the assumption of like for target movie by fifth person
    P(M|x of X_train) / sum = the assumption of dislike for target movie by fifth person
    """
    posteriors = []
    for x in X: # [1, 1, 0]
        #print("x: ",x)
        # posterior is proportional to prior * likelihood
        posterior = prior.copy() # Prior: {'Y': 0.75, 'N': 0.25}
        #print("posterior: ",posterior)
        for label, likelihood_label in likelihood.items(): # {'Y': array([0.4, 0.6, 0.4]), 'N': array([0.33333333, 0.33333333, 0.66666667])}
            #print("label and likelihood_label: ",label,likelihood_label)
            for index, bool_value in enumerate(x):
                #print("index and bool_value: ",index,bool_value)
                #print("likelihood_label[index]: ",likelihood_label[index])
                posterior[label] *= likelihood_label[index] if bool_value else (1 - likelihood_label[index])
                #print("posterior[label]: ",posterior[label])
        # normalize so that all sums up to 1
        #print("posterior.values(): ",posterior.values())
        sum_posterior = sum(posterior.values())
        #print("sum_posterior",sum_posterior)
        for label in posterior:
            if posterior[label] == float('inf'): # infimum
                posterior[label] = 1.0
            else:
                posterior[label] /= sum_posterior
        posteriors.append(posterior.copy())
    return posteriors

In [9]:
"""
Compute the posterior

Step 1: posterior[label] *= likelihood_label[index] if bool_value else (1 - likelihood_label[index])

Do for Y:
.75*.4 (because X_test[0] is 1, thus true) 
= 0.30000000000000004*.6(because X_test[1] is 1, thus true)
= 0.18000000000000002*(1-.4) (because X_test[2] is 0, thus false)
= 0.10800000000000001 = Y

Do for N:
.25*0.3333333333333333 (because X_test[0] is 1, thus true)
= 0.08333333333333333*0.3333333333333333 (because X_test[1] is 1, thus true)
= 0.027777777777777776*(1-0.6666666666666666)  (because X_test[2] is 0, thus false)
= 0.00925925925925926 = N

Step 2: sum_posterior = sum(posterior.values())
0.10800000000000001+0.00925925925925926 = 0.11725925925925927

Step 3: posterior[label] /= sum_posterior

Do for Y:
0.10800000000000001 / 0.11725925925925927 = 0.9210360075805433 = Y

Do for N:
0.00925925925925926 / 0.11725925925925927 = 0.07896399241945673 = N
"""

posterior = get_posterior(X_test, prior, likelihood)
print('Posterior:\n', posterior)

Posterior:
 [{'Y': 0.9210360075805433, 'N': 0.07896399241945673}]
